In [130]:
import spotipy
import json
import requests
from spotipy import util

# Spotify API

## Authentication

In [185]:
SPOTIPY_CLIENT_ID=''
SPOTIPY_CLIENT_SECRET=''
SPOTIPY_REDIRECT_URI='http://localhost'
USERNAME='militarpancho'
SCOPE='user-follow-read user-top-read'
SONGKICK_API_KEY = ""
TOP_ARTISTS_TIME_RANGE = "medium_term" #long_term|medium_term|short_term
LOCATION = "Madrid"

## Get User's Followed Artists

In [190]:
if token:
    sp = spotipy.Spotify(auth=token)
    # Artists followed
    artists_followed_raw = sp.current_user_followed_artists()
    artists_followed = artists_followed_raw['artists']['items']
    while artists_followed_raw['artists']['next'] != None:
        artists_followed_raw = json.loads(requests.get(artists_followed_raw['artists']['next'], headers = {'Authorization': 'Bearer ' + token}).text)
        artists_followed = artists_followed + artists_followed_raw['artists']['items']
    # User top artists 
    user_top_artists_raw = sp.current_user_top_artists(time_range=TOP_ARTISTS_TIME_RANGE)
    user_top_artists = user_top_artists_raw['items']
    while user_top_artists_raw['next'] != None:
        user_top_artists_raw = json.loads(requests.get(user_top_artists_raw['next'], headers = {'Authorization': 'Bearer ' + token}).text)
        user_top_artists = user_top_artists + user_top_artists_raw['items']

# Songkick API

In [191]:
songkick_endpoint = "https://api.songkick.com/api/3.0"
artists_followed_and_user_top_artists = artists_followed + user_top_artists
location_id = requests.get("{}/search/locations.json?query={}&apikey={}".format(songkick_endpoint,LOCATION,SONGKICK_API_KEY)).json()["resultsPage"]["results"]["location"][0]['metroArea']['id']
calendar = []
for i in artists_followed_and_user_top_artists:
    r = requests.get("{}/search/artists.json?apikey={}&query={}".format(songkick_endpoint, SONGKICK_API_KEY, i['name'])).json()
    artist_id = r['resultsPage']['results']['artist'][0]['id']
    similar_artists = requests.get("{}/artists/{}/similar_artists.json?apikey={}".format(songkick_endpoint, artist_id,SONGKICK_API_KEY)).json()['resultsPage']['results']
    event = requests.get("{}/events.json?apikey={}&location=sk:{}&artist_id={}".format(songkick_endpoint, SONGKICK_API_KEY, location_id, artist_id)).json()['resultsPage']['results']
    print "Artist_name: {}".format(i['name'])
    if event:
        print "One event found"
        calendar.append(event)
    if 'artist' in similar_artists:
        print "Looking for similar artists events..."
        events_count = 0
        for i in similar_artists['artist']:
            event = requests.get("{}/events.json?apikey={}&location=sk:{}&artist_id={}".format(songkick_endpoint, SONGKICK_API_KEY, location_id, i['id'])).json()['resultsPage']['results']
            if event:
                events_count += 1
                calendar.append(event)
        print "{} events found from similar artists".format(events_count)


Artist_name: LCD Soundsystem
Looking for similar artists events...
4 events found from similar artists
Artist_name: The Killers
Looking for similar artists events...
3 events found from similar artists
Artist_name: Kaiser Chiefs
One event found
Looking for similar artists events...
7 events found from similar artists
Artist_name: Los Planetas
Looking for similar artists events...
10 events found from similar artists
Artist_name: MGMT
Looking for similar artists events...
2 events found from similar artists
Artist_name: Wolf Parade
Looking for similar artists events...
0 events found from similar artists
Artist_name: David Bowie
Artist_name: Justice
Looking for similar artists events...
4 events found from similar artists
Artist_name: Cosmen Adelaida
Looking for similar artists events...
7 events found from similar artists
Artist_name: The Velvet Underground
Artist_name: Phoenix
Looking for similar artists events...
3 events found from similar artists
Artist_name: The Rolling Stones
Loo

KeyboardInterrupt: 